In [0]:
from google.cloud import storage
import tarfile
import numpy as np
from PIL import Image
import io
import tensorflow as tf
from matplotlib import pylab as plt

## Connect with GCS bucket

In [3]:
!export GOOGLE_APPLICATION_CREDENTIALS="./605_Final_Project-d51ffb7495d7.json"

def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

implicit()

DefaultCredentialsError: ignored

In [2]:
# Authentication
# !export GOOGLE_APPLICATION_CREDENTIALS="./605_Final_Project-d51ffb7495d7.json"
# !echo $GOOGLE_APPLICATION_CREDENTIALS
GOOGLE_APPLICATION_CREDENTIALS="./605_Final_Project-d51ffb7495d7.json"
bucket_name = "dataproc-staging-us-central1-759291875656-wohgf1sk"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

file_prefix = "data/"
blobs = bucket.list_blobs(prefix=file_prefix, delimiter = '/')

# list files in folder
for blob in blobs:
    print(blob.name)

DefaultCredentialsError: ignored

## Download file from blob

In [0]:
file_name = "val_data.tar"
blob = bucket.get_blob(file_prefix + file_name)
blob.download_to_filename("small_data.tar")

TEST_NUM_IMAGE = 10

image_byte = []
tar_file = tarfile.open("small_data.tar")

num = 0
for member in tar_file.getmembers():
  if num <= TEST_NUM_IMAGE:
    f = tar_file.extractfile(member)
    if f is not None:
        content = f.read()
        image_byte.append(content)
    num += 1
  else:
    break
        
tar_file.close()

In [0]:
data = image_byte[:TEST_NUM_IMAGE]

In [0]:
image = Image.open(io.BytesIO(data))
image

In [0]:
def resize_img(img, dim_h):
    """ 
  Resize image so that it has height dim_h 
  Args:
    img_path: (string) image path
    dim_h: (int) desired height
  Returns:
    img: (np array) the resized image
    """
    long = max(img.size)
    scale = dim_h/long
    img = img.resize((dim_h, dim_h), Image.ANTIALIAS)
#     img = kp_image.img_to_array(img)

    return img

In [0]:
resize_im = resize_img(image, 512)
resize_im_tf = tf.convert_to_tensor(np.array(resize_im))
# flatten image
im_flatten = tf.reshape(resize_im_tf,[1,-1])